In [24]:
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers
from datasets import load_dataset


In [25]:
ds = load_dataset('uoft-cs/cifar10')

In [26]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})


In [27]:
SEED = 42
EPOCHS = 30
LR = 1e-3
IMG_SIZE = (32, 32)

In [28]:
train = ds['train']
test = ds['test']

train = train.with_format('tensorflow').to_tf_dataset(
    columns=['img'],
    label_cols='label',
    batch_size=128,
    shuffle=True
)

test = test.with_format('tensorflow').to_tf_dataset(
    columns=['img'],
    label_cols='label',
    batch_size=128,
    shuffle=False
)

def ensure_shape(x, y):
    x = tf.cast(x, tf.float32)
    x.set_shape((None, IMG_SIZE[0], IMG_SIZE[1], 3))
    y = tf.cast(y, tf.int32)
    return x, y

train = train.map(ensure_shape, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
test = test.map(ensure_shape, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

/home/meu1404/miniforge/envs/rapids-ml/lib/python3.12/site-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [29]:
def build_model():
    inputs = keras.Input(shape=(*IMG_SIZE, 3))

    x = layers.Rescaling(1./255)(inputs)
    x = layers.RandomFlip("horizontal")(x)
    x = layers.RandomTranslation(0.125, 0.125, fill_mode="reflect")(x)

    def conv_block(x, filters, k=3, s=1, p="same"):
        x = layers.Conv2D(filters, k, strides=s, padding=p, use_bias=False,
                          kernel_initializer="he_normal")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    x = conv_block(x, 64);  x = conv_block(x, 64)
    x = layers.MaxPool2D()(x)

    x = conv_block(x, 128); x = conv_block(x, 128)
    x = layers.MaxPool2D()(x)

    x = conv_block(x, 256); x = conv_block(x, 256)
    x = layers.MaxPool2D()(x)

    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(256, 1, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(10, activation="softmax")(x)

    model = keras.Model(inputs, outputs, name="cifar10_cnn")
    return model

model = build_model()
model.summary()

Model: "cifar10_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_1 (RandomFlip)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation_1            │ (None, 32, 32, 3)      │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 64)     │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_8 (ReLU)                  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 128)    │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_9 (ReLU)                  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 128)    │       147,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_10 (ReLU)                 │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 256)      │       294,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_11 (ReLU)                 │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 8, 8, 256)      │       589,82

 Total params: 1,217,226 (4.64 MB)

 Trainable params: 1,214,922 (4.63 MB)

 Non-trainable params: 2,304 (9.00 KB)

In [30]:
opt = keras.optimizers.Adam(learning_rate=LR)
model.compile(
    optimizer=opt,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [31]:

callbacks = [
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, verbose=1, min_lr=1e-5
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=8, mode="max", restore_best_weights=True, verbose=1
    )
]

In [32]:
history = model.fit(
    train,
    validation_data=test,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/30


E0000 00:00:1757755938.488035    1771 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cifar10_cnn_1/dropout_2_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


391/391 ━━━━━━━━━━━━━━━━━━━━ 33s 77ms/step - acc: 0.5066 - loss: 1.3486 - val_acc: 0.5168 - val_loss: 1.4652 - learning_rate: 0.0010
Epoch 2/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 29s 75ms/step - acc: 0.6705 - loss: 0.9274 - val_acc: 0.6242 - val_loss: 1.1649 - learning_rate: 0.0010
Epoch 3/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 76ms/step - acc: 0.7387 - loss: 0.7476 - val_acc: 0.6357 - val_loss: 1.3381 - learning_rate: 0.0010
Epoch 4/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 29s 74ms/step - acc: 0.7780 - loss: 0.6429 - val_acc: 0.7215 - val_loss: 0.7926 - learning_rate: 0.0010
Epoch 5/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 75ms/step - acc: 0.8015 - loss: 0.5764 - val_acc: 0.7021 - val_loss: 0.9850 - learning_rate: 0.0010
Epoch 6/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 82ms/step - acc: 0.8188 - loss: 0.5258 - val_acc: 0.7996 - val_loss: 0.6173 - learning_rate: 0.0010
Epoch 7/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 78ms/step - acc: 0.8377 - loss: 0.4745 - val_acc: 0.7106 - val_loss: 0.9496 - learning_rate: 0.0010
Epo

2025-09-13 09:37:20.378243: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 1573376 bytes after encountering the first element of size 1573376 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 78ms/step - acc: 0.8667 - loss: 0.3879 - val_acc: 0.7552 - val_loss: 0.7695 - learning_rate: 0.0010
Epoch 11/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - acc: 0.8755 - loss: 0.3632 - val_acc: 0.8442 - val_loss: 0.4703 - learning_rate: 0.0010
Epoch 12/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - acc: 0.8814 - loss: 0.3441 - val_acc: 0.7962 - val_loss: 0.6367 - learning_rate: 0.0010
Epoch 13/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 82ms/step - acc: 0.8878 - loss: 0.3259 - val_acc: 0.8520 - val_loss: 0.4514 - learning_rate: 0.0010
Epoch 14/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 77ms/step - acc: 0.8955 - loss: 0.3044 - val_acc: 0.8404 - val_loss: 0.4737 - learning_rate: 0.0010
Epoch 15/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 33s 84ms/step - acc: 0.9021 - loss: 0.2824 - val_acc: 0.8126 - val_loss: 0.6264 - learning_rate: 0.0010
Epoch 16/30
390/391 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - acc: 0.9027 - loss: 0.2788
Epoch 16: ReduceLROnPlateau reducing learning rate to 0.000

In [33]:
test_loss, test_acc = model.evaluate(test, verbose=0)
print(f"Evaluate test_acc={test_acc:.4f}  test_loss={test_loss:.4f}")

Evaluate test_acc=0.9073  test_loss=0.3171
